In [1]:
from gurobipy import *
from prettytable import PrettyTable
import numpy as np

In [2]:
EC = Model('Dynamic Economic Dispatch')

In [3]:
#Sets
Gen = range(4)
T =  3# Demand per Period
Period = range(T)

GenAt = {Gen[0]:{'a':0.12 ,'b':14.80 ,'c':89, 'd':1.2 ,'e':-5 ,'f':3 , 'Pmin':28 ,'Pmax':200,'RU0':40,'RD0':40 },
      Gen[1]:{'a':0.17 ,'b':16.57 ,'c':83 , 'd':2.3 ,'e':-4.24 ,'f':6.09 , 'Pmin':20 ,'Pmax':290,'RU0':30,'RD0':30  },
      Gen[2]:{'a':0.15 ,'b':15.55 ,'c':100 , 'd':1.1 ,'e':-2.15 ,'f':5.69 ,'Pmin':30 ,'Pmax':190,'RU0':30,'RD0':30  },
      Gen[3]:{'a':0.19 ,'b':16.21 ,'c':70 , 'd':1.1 ,'e':-3.99 ,'f':6.2 ,'Pmin':20 ,'Pmax':260, 'RU0':50,'RD0':50 }  }

In [4]:
Demand = [ 200, 800, 300]

In [5]:
P = EC.addVars(Gen,Period)
uB = EC.addConstrs(P[(i, j)]  <= GenAt[i]['Pmax'] for i in Gen for j in Period)
lB = EC.addConstrs(P[(i, j)]  >= GenAt[i]['Pmin'] for i in Gen for j in Period)
#RU = EC.addConstrs((P[(i, j)] - P[(i, j-1)]  <= GenAt[i]['RU0']) for i in Gen for j in timePeriods)
#print(RU)
#RD = EC.addConstrs(P[(i, j)] - P[(i, j)]  <= GenAt[i]['RD0'] for i in Gen for j in Period)
Load = EC.addConstrs((P.sum('*',j) == Demand[j] for j in Period), name = 'DemandConst')
GenCost =quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i,j]+GenAt[i]['a']*P[i,j]*P[i,j] for i in Gen for j in Period) 
EC.setObjective(GenCost, GRB.MINIMIZE)

In [6]:
EC.setObjective(GenCost, GRB.MINIMIZE)
EC.Params.Method=-1
EC.setParam( 'OutputFlag', False )
EC.optimize()



status = EC.status
if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE \
or status == GRB.Status.UNBOUNDED:
    EC.computeIIS()
    EC.write("model.ilp")

Parameter Method unchanged
   Value: -1  Min: -1  Max: 5  Default: -1


In [7]:
PVar = np.asarray([[P[i,j].x for i in Gen] for j in Period])

In [8]:
Heading1 = ['Period']
Heading2 = ['Gen' + str(i) for i in Gen]
Headings = Heading1 + Heading2
Results = PrettyTable(Headings)
for j in Period:
    lis1 = ['Period ' + str(j)]
    lis2 = list(PVar[j,:])
    lis = lis1 +lis2
    Results.add_row(lis)              
print(Results)

+----------+--------------------+--------------------+--------------------+--------------------+
|  Period  |        Gen0        |        Gen1        |        Gen2        |        Gen3        |
+----------+--------------------+--------------------+--------------------+--------------------+
| Period 0 | 67.38454706927152  | 42.359680284191796 | 51.40763765541746  | 38.848134991119224 |
| Period 1 | 199.99999999999991 |  215.888888888889  | 189.99999999999977 | 194.11111111111126 |
| Period 2 | 99.25745016776077  | 64.85820011841045  | 76.90596013419841  | 58.97838957963039  |
+----------+--------------------+--------------------+--------------------+--------------------+


In [9]:
EC.printStats()


Statistics for model Dynamic Economic Dispatch :
  Linear constraint matrix    : 27 Constrs, 12 Vars, 36 NZs
  Matrix coefficient range    : [ 1, 1 ]
  Objective coefficient range : [ 14.8, 16.57 ]
  Variable bound range        : [ 0, 0 ]
  RHS coefficient range       : [ 20, 800 ]
